In [2]:
%load_ext autoreload
%autoreload

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Por sinal. Então, se você tem uma situação em que tem uma série de retornos, em que o retorno médio é menor que o retorno médio, o que isso significa? Isso significa que a mediana, que é o que costuma acontecer muito, é menor do que a média, e a média é o valor esperado. O que isso significa é que você tem retornos distorcidos negativamente. Vou comparar a média e a mediana.

In [4]:
## Importando a base de dados
## são os retornos, a partir de 1997,
## os retornos mensais de vários tipos de fundos de hedge.
hfi = pd.read_csv('/content/edhec-hedgefundindices.csv')
hfi.head()

,date,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
0,31/01/1997,1.19,3.93,1.78,7.91,1.89,2.13,1.91,5.73,2.81,1.50,1.80,-1.66,3.17
1,28/02/1997,1.23,2.98,1.22,5.25,1.01,0.84,1.22,1.75,-0.06,0.34,1.18,4.26,1.06
2,31/03/1997,0.78,-0.21,-0.12,-1.20,0.16,-0.23,1.09,-1.19,-0.84,0.60,0.10,7.78,-0.77
3,30/04/1997,0.86,-1.70,0.30,1.19,1.19,-0.05,1.30,1.72,0.84,-0.01,1.22,-1.29,0.09
4,31/05/1997,1.56,-0.15,2.33,3.15,1.89,3.46,1.18,1.08,3.94,1.97,1.73,-7.37,2.75


In [7]:
pd.concat([hfi.mean(),hfi.median(),hfi.mean()>hfi.median()], axis='columns')

<ipython-input-7-ad7338191468>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  pd.concat([hfi.mean(),hfi.median(),hfi.mean()>hfi.median()], axis='columns')
<ipython-input-7-ad7338191468>:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  pd.concat([hfi.mean(),hfi.median(),hfi.mean()>hfi.median()], axis='columns')
<ipython-input-7-ad7338191468>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated

,0,1,2
Convertible Arbitrage,0.550837,0.65,False
CTA Global,0.407376,0.14,True
Distressed Securities,0.694601,0.89,False
Emerging Markets,0.625323,0.96,False
Equity Market Neutral,0.449810,0.51,False
Event Driven,0.634449,0.84,False
Fixed Income Arbitrage,0.436502,0.55,False
Global Macro,0.540304,0.38,True
Long/Short Equity,0.633080,0.79,False
Merger Arbitrage,0.535589,0.60,False


 Veja isso, em praticamente todos os casos, vemos que a média é maior do que a mediana. O que isso está basicamente dizendo é que, além de CTA Global e Global macro, todas essas, e talvez vendas a descoberto, todas elas têm uma distorção negativa significativa.

$S(R)=\frac{E[(R-E(R))^{3}]}{\sigma^{3}_{R}}$



Então, o que estamos fazendo aqui? Então, a primeira coisa é que temos que calcular r menos o valor esperado de r. O valor esperado de r é a média de r. Então, essa é uma maneira elegante de dizer, rebaixar r. Então, pegamos a série de retorno e você produz r humilhado. É justo? Então, o que você precisa fazer é produzir essa segunda peça aqui e aquela segunda peça aqui é apenas o cubo da volatilidade. Então é isso que estou fazendo aqui. O sigma r é a volatilidade. Se você se lembra, tínhamos essa distinção entre a população e a amostra. Então, estou enviando que ddof é igual a zero, o que significa que os graus de liberdade foram definidos como zero, o que basicamente significa que não faça essa correção n menos uma. Realmente não importa se você tem dados suficientes, então dividir por n ou n menos um resultará em números ligeiramente diferentes.

In [20]:
def skewness(r):
    demeaned_r = r - r.mean()
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**3).mean()
    return exp/sigma_r**3

In [9]:
skewness(hfi).sort_values()

<ipython-input-8-9a7f874393da>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  demeaned_r = r - r.mean()
<ipython-input-8-9a7f874393da>:3: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  sigma_r = r.std(ddof=0)


Fixed Income Arbitrage   -3.940320
Convertible Arbitrage    -2.639592
Equity Market Neutral    -2.124435
Relative Value           -1.815470
Event Driven             -1.409154
Merger Arbitrage         -1.320083
Distressed Securities    -1.300842
Emerging Markets         -1.167067
Long/Short Equity        -0.390227
Funds Of Funds           -0.361783
CTA Global                0.173699
Short Selling             0.767975
Global Macro              0.982922
date                           NaN
dtype: float64

In [10]:
import scipy.stats

In [22]:
hfi_numeric = hfi.apply(pd.to_numeric, errors='coerce')
scipy.stats.skew(hfi_numeric)

array([        nan, -2.63959223,  0.17369864, -1.30084204, -1.16706749,
       -2.12443538, -1.40915356, -3.94032029,  0.98292188, -0.39022677,
       -1.32008333, -1.81546975,  0.76797484, -0.36178308])

Vamos gerar retornos de, digamos, uma média de zero, desvio padrão de 15% e vamos gerar exatamente o mesmo número de retornos que tivemos aqui. Só para que possamos compará-lo.

In [23]:
import numpy as np

In [24]:
normal_rets = np.random.normal(0, .15, size=(26300,1))

In [25]:
skewness(normal_rets)

-0.00860769352372155

$K(R)=\frac{E[R-E(R)^{4}]}{\sigma_{R}^{4}}$

In [26]:
def kurtosis(r):
    demeaned_r = r - r.mean()
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**4).mean()
    return exp/sigma_r**4

In [27]:
kurtosis(normal_rets)

3.050298302146261

In [29]:
kurtosis(hfi)

<ipython-input-26-f41a38c79fed>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  demeaned_r = r - r.mean()
<ipython-input-26-f41a38c79fed>:3: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  sigma_r = r.std(ddof=0)


CTA Global                 2.952960
Convertible Arbitrage     23.280834
Distressed Securities      7.889983
Emerging Markets           9.250788
Equity Market Neutral     17.218555
Event Driven               8.035828
Fixed Income Arbitrage    29.842199
Funds Of Funds             7.070153
Global Macro               5.741679
Long/Short Equity          4.523893
Merger Arbitrage           8.738950
Relative Value            12.121208
Short Selling              6.117772
date                            NaN
dtype: float64

In [31]:
hfi_numeric = hfi.apply(pd.to_numeric, errors='coerce')
scipy.stats.kurtosis(hfi_numeric)

array([        nan, 20.28083446, -0.04703963,  4.88998336,  6.25078841,
       14.21855526,  5.03582817, 26.84219928,  2.74167945,  1.52389258,
        5.73894979,  9.12120787,  3.11777175,  4.07015278])

In [32]:
import scipy.stats
def is_normal(r, level=0.01):
    if isinstance(r, pd.DataFrame):
        return r.aggregate(is_normal)
    else:
        statistic, p_value = scipy.stats.jarque_bera(r)
        return p_value > level

In [34]:
hfi_numeric = hfi.apply(pd.to_numeric, errors='coerce')
is_normal(hfi_numeric)

date                      False
Convertible Arbitrage     False
CTA Global                 True
Distressed Securities     False
Emerging Markets          False
Equity Market Neutral     False
Event Driven              False
Fixed Income Arbitrage    False
Global Macro              False
Long/Short Equity         False
Merger Arbitrage          False
Relative Value            False
Short Selling             False
Funds Of Funds            False
dtype: bool